<a href="https://colab.research.google.com/github/KTSNVaishnavi/Machine-Learning/blob/main/Neural_Networks_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [348]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [349]:
!pip install collinearity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [350]:
import pandas as pd
import numpy as np
#For detecting the low variance
from sklearn.feature_selection import VarianceThreshold
#For detecting the outliers
from scipy import stats
#For detecting the collinearity 
from collinearity import SelectNonCollinear
#For scaling the data, and encoding
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV
#For class imbalance
from imblearn.over_sampling import SMOTE, ADASYN 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [351]:
#Reading the data
data=pd.read_csv('/content/gdrive/MyDrive/MachineLearning/BackOrders.csv')

In [352]:
data.shape

(61589, 23)

In [353]:
data.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

In [354]:
data.head()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1888279,117,NaN,0,0,0,0,0,0,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,1870557,7,2.0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,1475481,258,15.0,10,10,77,184,46,132,256,...,0,0.54,0.70,0,No,No,No,Yes,No,No
3,1758220,46,2.0,0,0,0,0,1,2,6,...,0,0.75,0.90,0,Yes,No,No,Yes,No,No
4,1360312,2,2.0,0,4,6,10,2,2,5,...,0,0.97,0.92,0,No,No,No,Yes,No,No


In [355]:
data.describe()

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
count,6.158900e+04,61589.000000,58186.000000,61589.000000,6.158900e+04,6.158900e+04,6.158900e+04,61589.000000,61589.000000,6.158900e+04,6.158900e+04,61589.000000,61589.000000,61589.000000,61589.000000,61589.000000
mean,2.037188e+06,287.721882,7.559619,30.192843,1.692728e+02,3.150413e+02,4.535760e+02,44.742957,150.732631,2.835465e+02,4.196427e+02,43.087256,1.605400,-6.264182,-5.863664,1.205361
std,6.564178e+05,4233.906931,6.498952,792.869253,5.286742e+03,9.774362e+03,1.420201e+04,1373.805831,5224.959649,8.872270e+03,1.269858e+04,959.614135,42.309229,25.537906,24.844514,29.981155
min,1.068628e+06,-2999.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,-99.000000,-99.000000,0.000000
25%,1.498574e+06,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.620000,0.640000,0.000000
50%,1.898033e+06,10.000000,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000,4.000000e+00,6.000000e+00,0.000000,0.000000,0.820000,0.800000,0.000000
75%,2.314826e+06,57.000000,8.000000,0.000000,1.200000e+01,2.500000e+01,3.600000e+01,6.000000,17.000000,3.400000e+01,5.100000e+01,3.000000,0.000000,0.960000,0.950000,0.000000
max,3.284895e+06,673445.000000,52.000000,170976.000000,1.126656e+06,2.094336e+06,3.062016e+06,295197.000000,934593.000000,1.799099e+06,2.631590e+06,192978.000000,7392.000000,1.000000,1.000000,2999.000000


In [356]:
#Checking the unique values in each variable for identifying the waste columns
data.nunique()

sku                  61589
national_inv          2916
lead_time               28
in_transit_qty         908
forecast_3_month      1623
forecast_6_month      2195
forecast_9_month      2664
sales_1_month         1092
sales_3_month         1928
sales_6_month         2679
sales_9_month         3220
min_bank              1098
potential_issue          2
pieces_past_due        190
perf_6_month_avg       102
perf_12_month_avg      102
local_bo_qty           201
deck_risk                2
oe_constraint            2
ppap_risk                2
stop_auto_buy            2
rev_stop                 2
went_on_backorder        2
dtype: int64

In [357]:
data=data.drop('sku',axis=1)

In [358]:
data.isnull().sum()

national_inv            0
lead_time            3403
in_transit_qty          0
forecast_3_month        0
forecast_6_month        0
forecast_9_month        0
sales_1_month           0
sales_3_month           0
sales_6_month           0
sales_9_month           0
min_bank                0
potential_issue         0
pieces_past_due         0
perf_6_month_avg        0
perf_12_month_avg       0
local_bo_qty            0
deck_risk               0
oe_constraint           0
ppap_risk               0
stop_auto_buy           0
rev_stop                0
went_on_backorder       0
dtype: int64

In [359]:
#Dropping the lead_time as it has only 5% of null values
data=data.drop('lead_time',axis=1)

In [360]:
#Checking the outliers
num=data.select_dtypes(['float64','int64']).columns
z = np.abs(stats.zscore(data[num]))
print(z)

       national_inv  in_transit_qty  forecast_3_month  forecast_6_month  \
0          0.040323        0.038081          0.032019          0.032232   
1          0.066304        0.038081          0.032019          0.032232   
2          0.007020        0.025468          0.030127          0.024354   
3          0.057092        0.038081          0.032019          0.032232   
4          0.067485        0.038081          0.031262          0.031618   
...             ...             ...               ...               ...   
61584      0.066540        0.038081          0.027479          0.029776   
61585      0.037252        0.038081          0.024452          0.024047   
61586      0.036071        0.038081          0.032019          0.032232   
61587      0.053077        0.038081          0.032019          0.032232   
61588      0.067957        0.038081          0.030505          0.031413   

       forecast_9_month  sales_1_month  sales_3_month  sales_6_month  \
0              0.031938    

In [361]:
# Position of the outlier
print(np.where(z > 3.5))
print(np.where(z < -3.5))

(array([    0,     0,    10, ..., 61581, 61587, 61587]), array([11, 12, 11, ..., 12, 11, 12]))
(array([], dtype=int64), array([], dtype=int64))


In [362]:
#There is no outlier
data= data[(z < 3.5).all(axis=1)]

In [363]:
data.shape

(56800, 21)

In [364]:
data.head()

,national_inv,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,sales_9_month,min_bank,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
1,7,0,0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,258,10,10,77,184,46,132,256,365,47,...,0,0.54,0.70,0,No,No,No,Yes,No,No
3,46,0,0,0,0,1,2,6,9,1,...,0,0.75,0.90,0,Yes,No,No,Yes,No,No
4,2,0,4,6,10,2,2,5,6,0,...,0,0.97,0.92,0,No,No,No,Yes,No,No
5,297,0,0,0,0,5,6,44,57,3,...,0,0.58,0.75,0,Yes,No,No,Yes,No,No


In [365]:
#Finding the collinear columns
cor_matrix = data.corr().abs()
print(cor_matrix)

                   national_inv  in_transit_qty  forecast_3_month  \
national_inv           1.000000        0.295015          0.328830   
in_transit_qty         0.295015        1.000000          0.603760   
forecast_3_month       0.328830        0.603760          1.000000   
forecast_6_month       0.366461        0.633622          0.956406   
forecast_9_month       0.374603        0.646219          0.938732   
sales_1_month          0.370190        0.685275          0.792950   
sales_3_month          0.387943        0.691395          0.801046   
sales_6_month          0.413110        0.687814          0.795899   
sales_9_month          0.418773        0.685710          0.788740   
min_bank               0.403451        0.660879          0.757156   
pieces_past_due        0.003493        0.060412          0.118576   
perf_6_month_avg       0.052833        0.043532          0.024189   
perf_12_month_avg      0.051223        0.044061          0.024082   
local_bo_qty           0.004376   

In [366]:
#Building an upper triangular matrix to see the correaltion of the columns more clearly 
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype('bool'))
print(upper_tri)

                   national_inv  in_transit_qty  forecast_3_month  \
national_inv                NaN        0.295015           0.32883   
in_transit_qty              NaN             NaN           0.60376   
forecast_3_month            NaN             NaN               NaN   
forecast_6_month            NaN             NaN               NaN   
forecast_9_month            NaN             NaN               NaN   
sales_1_month               NaN             NaN               NaN   
sales_3_month               NaN             NaN               NaN   
sales_6_month               NaN             NaN               NaN   
sales_9_month               NaN             NaN               NaN   
min_bank                    NaN             NaN               NaN   
pieces_past_due             NaN             NaN               NaN   
perf_6_month_avg            NaN             NaN               NaN   
perf_12_month_avg           NaN             NaN               NaN   
local_bo_qty                NaN   

In [367]:
#View the columns with high collinearity
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
print(); print(to_drop)


['forecast_6_month', 'forecast_9_month', 'sales_3_month', 'sales_6_month', 'sales_9_month']


In [368]:
data = data.drop(data[to_drop], axis=1)
print(); print(data.head())


   national_inv  in_transit_qty  forecast_3_month  sales_1_month  min_bank  \
1             7               0                 0              0         0   
2           258              10                10             46        47   
3            46               0                 0              1         1   
4             2               0                 4              2         0   
5           297               0                 0              5         3   

  potential_issue  pieces_past_due  perf_6_month_avg  perf_12_month_avg  \
1              No                0              0.50               0.28   
2              No                0              0.54               0.70   
3              No                0              0.75               0.90   
4              No                0              0.97               0.92   
5              No                0              0.58               0.75   

   local_bo_qty deck_risk oe_constraint ppap_risk stop_auto_buy rev_stop  \
1  

In [369]:
data.shape

(56800, 16)

In [370]:
#Converting the data types 
cat=data.select_dtypes(include='object').astype('category').columns
data[cat]=data[cat].astype('category')
data.dtypes


national_inv            int64
in_transit_qty          int64
forecast_3_month        int64
sales_1_month           int64
min_bank                int64
potential_issue      category
pieces_past_due         int64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty            int64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
went_on_backorder    category
dtype: object

In [371]:
#Separating the num cols
num=data.select_dtypes(include=['int64','float64']).columns
data[num].astype('float64')


,national_inv,in_transit_qty,forecast_3_month,sales_1_month,min_bank,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty
1,7.0,0.0,0.0,0.0,0.0,0.0,0.50,0.28,0.0
2,258.0,10.0,10.0,46.0,47.0,0.0,0.54,0.70,0.0
3,46.0,0.0,0.0,1.0,1.0,0.0,0.75,0.90,0.0
4,2.0,0.0,4.0,2.0,0.0,0.0,0.97,0.92,0.0
5,297.0,0.0,0.0,5.0,3.0,0.0,0.58,0.75,0.0
...,...,...,...,...,...,...,...,...,...
61583,80.0,0.0,0.0,7.0,3.0,0.0,1.00,0.91,0.0
61584,6.0,0.0,24.0,0.0,1.0,0.0,0.98,0.98,0.0
61585,130.0,0.0,40.0,18.0,34.0,0.0,0.51,0.28,0.0
61586,135.0,0.0,0.0,10.0,2.0,0.0,1.00,0.99,0.0


In [372]:
#Removing the low variance
from sklearn.feature_selection import VarianceThreshold
var_thr = VarianceThreshold(threshold = 0.25) #Removing both constant and quasi-constant
var_thr.fit(data[num])
var_thr.get_support()

array([ True,  True,  True,  True,  True,  True, False, False,  True])

In [373]:
concol = [column for column in data[num].columns 
          if column not in data[num].columns[var_thr.get_support()]]

for features in concol:
    print(features)

perf_6_month_avg
perf_12_month_avg


In [374]:
num=num.drop(concol)
num

Index(['national_inv', 'in_transit_qty', 'forecast_3_month', 'sales_1_month',
       'min_bank', 'pieces_past_due', 'local_bo_qty'],
      dtype='object')

In [375]:
data.shape

(56800, 16)

In [376]:
#Train and Test Split
X_train,X_test,y_train,y_test=train_test_split(data.loc[:, data.columns!='went_on_backorder'], data['went_on_backorder'], test_size=0.3, random_state=100,stratify=data['went_on_backorder'])

In [377]:
#Checking the value counts of the target variable
y_train.value_counts()

No     32245
Yes     7515
Name: went_on_backorder, dtype: int64

In [378]:
cat=cat.drop('went_on_backorder')
X_train_num=X_train[num]
X_test_num=X_test[num]
X_train_cat=X_train[cat]
X_test_cat=X_test[cat]


In [379]:
#Imputing the null values
sim_num=SimpleImputer(strategy='mean')
sim_cat=SimpleImputer(strategy='most_frequent')
X_train_num=pd.DataFrame(sim_num.fit_transform(X_train_num),columns=X_train_num.columns)
X_test_num=pd.DataFrame(sim_num.transform(X_test_num),columns=X_test_num.columns)
X_train_cat=pd.DataFrame(sim_cat.fit_transform(X_train_cat),columns=X_train_cat.columns)
X_test_cat=pd.DataFrame(sim_cat.transform(X_test_cat),columns=X_test_cat.columns)

In [380]:
X_train_num.isnull().sum()
X_train_cat.isnull().sum()

potential_issue    0
deck_risk          0
oe_constraint      0
ppap_risk          0
stop_auto_buy      0
rev_stop           0
dtype: int64

In [381]:
#Standardizing the data
std=StandardScaler()
X_train_num=pd.DataFrame(std.fit_transform(X_train_num),columns=X_train_num.columns)
X_test_num=pd.DataFrame(std.transform(X_test_num),columns=X_test_num.columns)

In [382]:
# #Initiating PCA to reduce dimentions aka features to 3
# pca = PCA(n_components=5)
# pca.fit(X_train[num])
# PCA_train = pd.DataFrame(pca.fit_transform(X_train[num]), columns=(["col1","col2", "col3","col4","col5"]))
# PCA_train.describe()

In [383]:
# pca = PCA(n_components=5)
# pca.fit(X_test[num])
# PCA_test = pd.DataFrame(pca.transform(X_test[num]), columns=(["col1","col2", "col3","col4","col5"]))
# PCA_test.describe()

In [384]:
X_train_cat

,potential_issue,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop
0,No,No,No,Yes,Yes,No
1,No,No,No,No,Yes,No
2,No,No,No,No,Yes,No
3,No,No,No,No,Yes,No
4,No,No,No,No,Yes,No
...,...,...,...,...,...,...
39755,No,No,No,No,Yes,No
39756,No,No,No,No,Yes,No
39757,No,Yes,No,No,No,No
39758,No,No,No,No,Yes,No


In [385]:
#Encoding the categorical columns
ohe=OneHotEncoder()
X_train_cat=X_train_cat.astype('category')
X_test_cat=X_test_cat.astype('category')
X_train_cat=pd.DataFrame(ohe.fit_transform(X_train_cat).todense(),columns=ohe.get_feature_names_out())
X_test_cat=pd.DataFrame(ohe.fit_transform(X_test_cat).todense(),columns=ohe.get_feature_names_out())
X_train_cat



,potential_issue_No,potential_issue_Yes,deck_risk_No,deck_risk_Yes,oe_constraint_No,oe_constraint_Yes,ppap_risk_No,ppap_risk_Yes,stop_auto_buy_No,stop_auto_buy_Yes,rev_stop_No,rev_stop_Yes
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
39755,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
39756,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
39757,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
39758,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [386]:
#Encoding the target variable
from sklearn.preprocessing import LabelEncoder
lab=LabelEncoder()
y_train=lab.fit_transform(y_train)
y_test=lab.transform(y_test)
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [387]:
#joining the cat and num
X_train=pd.concat([X_train_num.reset_index(drop=True),X_train_cat.reset_index(drop=True)],axis=1)
X_test=pd.concat([X_test_num.reset_index(drop=True),X_test_cat.reset_index(drop=True)],axis=1)
X_train.isnull().sum()
X_train.shape

(39760, 19)

In [388]:
#Checking the value counts of the target variable
np.unique(y_train,return_counts=True)

(array([0, 1]), array([32245,  7515]))

In [389]:
#There is a class imbalance
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state=30)
X_train,y_train=sm.fit_resample(X_train,y_train.ravel())
X_test,y_test=sm.fit_resample(X_test,y_test.ravel())
np.unique(y_train,return_counts=True)

(array([0, 1]), array([23671, 25658]))

In [390]:
#Model building
model=LogisticRegressionCV(max_iter=1000,penalty='l2',solver='newton-cg',class_weight='balanced')
log=model.fit(X_train,y_train.ravel())

In [391]:
#Metrics
train_pred=log.predict(X_train)
test_pred=log.predict(X_test)

In [392]:
print(confusion_matrix(train_pred,y_train))
print(confusion_matrix(test_pred,y_test))
print(classification_report(train_pred,y_train))
print(classification_report(test_pred,y_test))

[[18870  1460]
 [ 4801 24198]]
[[ 8096   662]
 [ 2454 10256]]
              precision    recall  f1-score   support

           0       0.80      0.93      0.86     20330
           1       0.94      0.83      0.89     28999

    accuracy                           0.87     49329
   macro avg       0.87      0.88      0.87     49329
weighted avg       0.88      0.87      0.87     49329

              precision    recall  f1-score   support

           0       0.77      0.92      0.84      8758
           1       0.94      0.81      0.87     12710

    accuracy                           0.85     21468
   macro avg       0.85      0.87      0.85     21468
weighted avg       0.87      0.85      0.86     21468



In [393]:
X_train.shape

(49329, 19)

In [394]:

#Building the neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense

In [395]:
model=Sequential()
model.add(Dense(10,input_dim=19, activation='relu'))
model.add(Dense(5,activation='relu'))       
model.add(Dense(1, activation='sigmoid'))  

In [396]:
from tensorflow.keras.optimizers import SGD 
customized_optimizer=SGD(learning_rate=0.01)

In [397]:
model.compile(loss='binary_crossentropy',optimizer=customized_optimizer,metrics=['accuracy'])

In [398]:
history = model.fit(X_train, y_train.ravel(),
                    batch_size=32,
                    validation_split=0.2,
                    epochs= 30)

Epoch 1/30
1234/1234 [==============================] - 3s 2ms/step - loss: 0.6671 - accuracy: 0.6088 - val_loss: 0.8931 - val_accuracy: 0.1099
Epoch 2/30
1234/1234 [==============================] - 3s 2ms/step - loss: 0.6436 - accuracy: 0.6306 - val_loss: 0.8973 - val_accuracy: 0.1368
Epoch 3/30
1234/1234 [==============================] - 3s 2ms/step - loss: 0.6026 - accuracy: 0.6648 - val_loss: 0.8337 - val_accuracy: 0.2119
Epoch 4/30
1234/1234 [==============================] - 3s 2ms/step - loss: 0.5504 - accuracy: 0.6950 - val_loss: 0.6709 - val_accuracy: 0.3221
Epoch 5/30
1234/1234 [==============================] - 3s 2ms/step - loss: 0.5061 - accuracy: 0.7325 - val_loss: 0.6651 - val_accuracy: 0.3688
Epoch 6/30
1234/1234 [==============================] - 3s 2ms/step - loss: 0.4670 - accuracy: 0.7797 - val_loss: 0.8065 - val_accuracy: 0.3167
Epoch 7/30
1234/1234 [==============================] - 3s 2ms/step - loss: 0.4342 - accuracy: 0.8097 - val_loss: 0.2584 - val_accuracy:

In [399]:
print(model.evaluate(X_train,y_train))
print(model.evaluate(X_test,y_test))

1542/1542 [==============================] - 3s 2ms/step - loss: 0.2325 - accuracy: 0.9356
[0.23251694440841675, 0.9356362223625183]
671/671 [==============================] - 1s 2ms/step - loss: 0.2359 - accuracy: 0.9390
[0.23586179316043854, 0.9389789700508118]


In [400]:
y_train_Pred = (model.predict(X_train)>0.5).astype("int32")
y_test_Pred = (model.predict(X_test)>0.5).astype("int32")

671/671 [==============================] - 1s 1ms/step


In [401]:
# Function for Classification Report
from sklearn.metrics import accuracy_score,classification_report
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [402]:
classifcation_report_train_test(y_train, y_train_Pred, y_test, y_test_Pred)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           0     0.9193    0.9492    0.9340     23671
           1     0.9517    0.9231    0.9372     25658

    accuracy                         0.9356     49329
   macro avg     0.9355    0.9362    0.9356     49329
weighted avg     0.9361    0.9356    0.9357     49329


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           0     0.9217    0.9572    0.9391     10550
           1     0.9570    0.9214    0.9389     10918

    accuracy                         0.9390     21468
   macro avg     0.9393    0.9393    0.9390     21468
weighted avg     0.9396    0.9390    0.9390     21468

